## Get the Metadata for the 990 IRS files

In [1]:
import os, sys
from joblib import Parallel, delayed

# METHOD 1
#import import_ipynb
#from api.classy990downloads import *

# METHOD 2 - This will reload without restarting notebook (debugging)
%run ../api/classy990downloads.ipynb

importing Jupyter notebook from /home/notebooks/api/filter_text.ipynb


[nltk_data] Downloading package wordnet to /home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#Set the YEAR of IRS 990 data that we want to download

year = 2017

In [3]:
# Load the object that parses the IRS 990 index and will provide
# each individual form to us

#documents = classy990forms(config='validElements.conf', mongo='mongodb://mongo/irs990', debug=False)
documents = classy990forms(mongo='mongodb://mongo/irs990', dumpmongo=False, debug=True)
documents.year( year )
documents

Working on year: 2017
index URL: 2017
Prepare INDEX on year: 2017
Found local copy: /home/notebooks/IRSX_CACHE//IRS990_2017.pkl
Init MongoDB inside classy990forms object [mongodb://mongo/irs990] [2017]
Connect to: [mongodb://mongo/irs990]
List of databases in MongoDB: ['admin', 'config', 'irs990', 'local']
Open db: [irs990]
Collection: [2017]
101 documents present


IRS 990 form for year [2017]: 489013 documents in index

In [4]:
#%run ../api/classy990downloads.ipynb
#documents = classy990forms(mongo='mongodb://mongo/irs990', debug=True)
#documents.year( year )
#documents.get("201612449349300786")

In [5]:
#%run ../api/classy990downloads.ipynb
#from irsx.xmlrunner import XMLRunner
#xml_runner = XMLRunner()
#activeRaw = xml_runner.run_filing( "201612449349300786", verbose=True )

In [6]:
#x
#activeRaw.get_parsed_sked('IRS990ScheduleO')
#activeRaw.get_result()

In [7]:
def readDoc(x):
    try:
        sys.stdout.write('Dowload EIN:%s\r' % x.index)
        sys.stdout.flush() 
        x.to_mongo()
    except:
        print('oops')

In [8]:
# Loop over all forms and upload to the database.
num_cores=1
Parallel(n_jobs=num_cores)(delayed(readDoc)(x) for x in documents)

ERROR: Problem on creating new Filing object
       Skipping this instance and continue with next
       
XML Parse error in /home/notebooks/IRSX_CACHE/201612449349300606_public.xml
File may be damaged or incomplete.
Try erasing this file and downloading again.
oops
ERROR: Problem on creating new Filing object
       Skipping this instance and continue with next
       
XML Parse error in /home/notebooks/IRSX_CACHE/201612519349300766_public.xml
File may be damaged or incomplete.
Try erasing this file and downloading again.
oops


KeyboardInterrupt: 